In [12]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import math
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
import numpy as np

In [19]:
def get_predictions_by_column(data, month):
    
    adf_test = adfuller(data)
    
    training_data_len = math.ceil(len(data) * .8)
    train = data[0:training_data_len]
    test = data[training_data_len:]

    model = SARIMAX(train, 
                order = (0, 1, 0), 
                seasonal_order = (0, 1, 0, 12))
    result = model.fit()
    
    start_test = len(train)
    end_test = len(train) + len(test) - 1
    predictions = result.predict(start_test, end_test)
        
    start = len(data)
    end = (len(data) - 1) + month
    forecast = result.predict(start, end)
    
    metrics = []
    metrics.append(mean_squared_error(test, predictions))
    metrics.append(np.sqrt(mean_squared_error(test, predictions)))
        
    return forecast, adf_test[1], metrics

In [20]:
def get_table_of_predictions(data, month):
    
    d_of_values = {}
    count = len(data.columns.tolist())
    
    for i in range(1, count):
        res, adf, metrics = get_predictions_by_column(data.iloc[:,i], month)
        d_of_values[data.columns.tolist()[i]] = res
        print(f'For {i} dataset p-value = {adf}, MSE = {metrics[0]}, RMSE = {metrics[1]}')
    
    df = pd.DataFrame(d_of_values) 
    
    return df

In [21]:
data = pd.read_csv('Общий.csv')
month = 17
df = get_table_of_predictions(data, month)
df

For 1 dataset p-value = 0.5202987796002672, MSE = 46653.44460302958, RMSE = 215.99408464823657
For 2 dataset p-value = 0.9990879026048354, MSE = 3091528.9469696977, RMSE = 1758.2744231119605
For 3 dataset p-value = 0.7971746391473259, MSE = 143.79459848338183, RMSE = 11.991438549372708
For 4 dataset p-value = 0.052300213164295034, MSE = 1283.0959181818164, RMSE = 35.82032828132395
For 5 dataset p-value = 0.99517341241929, MSE = 3841.8134090909934, RMSE = 61.98236369396535
For 6 dataset p-value = 0.0586654535466985, MSE = 217.76009772673305, RMSE = 14.756696707824995


,AFLT,LKOH,MAGN,SBER,SIBN,SNGS
169,460.01,3284.0,67.490,280.95,372.85,10.825
170,462.81,3179.0,67.145,284.75,347.55,10.880
171,470.56,3016.0,64.415,290.15,344.20,9.340
172,474.78,2834.0,66.400,299.16,348.90,9.045
173,478.58,2948.0,64.680,299.60,353.15,8.445
174,478.78,3131.0,70.175,305.75,357.60,6.445
175,491.73,3177.0,71.195,310.10,363.90,6.430
176,510.01,3332.0,69.240,311.94,368.70,5.950
177,523.03,3364.0,72.540,314.00,376.50,2.255
178,527.91,3452.0,75.125,325.30,391.70,5.210
